In [ ]:
from torch import nn
from transformers import BertModel

In [ ]:
class BERTClassifier(nn.Module):
    
  def __init__(self):
      
    # Specify network layers
    super(BERTClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    
    self.avg_pool = nn.AdaptiveAvgPool1d(1)
    
    self.linear = nn.Linear(self.bert.config.hidden_size, 1)
    
    # Define dropout
    self.dropout = nn.Dropout(0.1)
    
    # Freeze BERT layers
    for n, p in self.bert.named_parameters():
      p.requires_grad = False

  def forward(self, text, masks):
    
    output_bert = self.bert(text, attention_mask=masks).last_hidden_state
    output_bert = self.avg_pool(output_bert.transpose(1, 2)).squeeze(-1)
    
    return self.linear(self.dropout(output_bert))
  


In [ ]:
from transformers import AutoTokenizer
import torch

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BERTClassifier()

#####################################################
#### Specify the path to the model state dict!!! ####
#####################################################
model.load_state_dict(torch.load("model/v8/bert_classifier.pth"))

model.eval()

# Input text
input_text = "The meeting sucks. I hate it."

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Perform inference
with torch.no_grad():
    outputs = model(inputs['input_ids'], masks=inputs['attention_mask'])

# Get the predicted class
logits = outputs
probability = torch.sigmoid(logits).item()
print(probability)

# Threshold the probability to get the predicted class
predicted_class = 1 if probability > 0.5 else 0

# Map to label
label_map = {0: "not toxic", 1: "toxic"}
predicted_label = label_map[predicted_class]

print(f"Predicted label: {predicted_label}")

/var/folders/t8/rdxqv77j1bz5zsm7dd6c_szh0000gn/T/ipykernel_69628/1137289105.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model/v8/be

0.7892874479293823
Predicted label: hate speech
